https://aws-data-wrangler.readthedocs.io/en/stable/stubs/awswrangler.s3.read_csv.html

In [1]:
pip install awswrangler

     |████████████████████████████████| 183 kB 14.6 MB/s eta 0:00:01
     |████████████████████████████████| 91 kB 3.0 MB/s  eta 0:00:01
     |████████████████████████████████| 43 kB 3.4 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import sagemaker
import boto3
import awswrangler as wr

import pandas as pd
import numpy as np
from datetime import datetime

import json

In [3]:
input_bucket = 'adsb-data'
input_subfolder = ''

output_bucket = 'partly-cloudy-common-area'
output_subfolder = ''

In [4]:
from sagemaker import get_execution_role
role = get_execution_role()

In [ ]:
# Examine the files in the 'adsb-data s3 Bucket
conn = boto3.client('s3')
contents = conn.list_objects(Bucket= input_bucket, Prefix= input_subfolder)['Contents']
for f in contents:
    print(f['Key'])

In [ ]:
# Examine the contents in the 'adsb-data' s3 Bucket
contents

In [ ]:
# Get all file names in 'adsb-data' Bucket into a list 
myFileNames = [f['Key'] for f in contents] # <---------------------- Using 'AWS contents' does not work!!!
# myFileNames = ['2016_06_20_0002Z.json', '2016_06_20_0003Z.json']

In [5]:
# Generate all file names assume to be in 'adsb-data' <--------------------------------- Redo by Iteration
dateTimes = pd.date_range('2016-06-20 00:00:00', '2016-06-20 23:59:00', freq= 'T')
dateTimesSer = pd.Series([str(dateTime) for dateTime in dateTimes], name= 'temp')
myFileNames = [dateTimesSer[i][0:4] + '_' + dateTimesSer[i][5:7] + '_' + dateTimesSer[i][8:10] + '_' + dateTimesSer[i][11:13] + dateTimesSer[i][14:16] +'Z.json' for i in range(len(dateTimesSer))]

In [ ]:
myFileNames

In [6]:
%%time
# Obtain necessary data items from all json files in 'adsb-data' in a single dataframe

dt = []
eyedee = []
alt = []
lat = []
long = []

for myFileName in myFileNames:
    s3_obj = boto3.client('s3')
    s3_myobj = s3_obj.get_object(Bucket= input_bucket, Key= myFileName)
    s3_mydata = s3_myobj['Body'].read().decode('utf-8')
    js = json.loads(s3_mydata)
    
    dattim = datetime(year= int(myFileName[0:4]),
                      month= int(myFileName[5:7]),
                      day= int(myFileName[8:10]),
                      hour= int(myFileName[11:13]),
                      minute= int(myFileName[13:15]),
                      second= 0)
    
    for i in range(len(js['acList'])):
        dt.append(str(dattim))
        if "Id" in js['acList'][i].keys():
            eyedee.append(js['acList'][i]['Id'])
        else:
            eyedee.append(np.nan)
        if "Alt" in js['acList'][i].keys():
            alt.append(js['acList'][i]['Alt'])
        else:
            alt.append(np.nan)
        if ("Lat" in js['acList'][i].keys()):
            if type(js['acList'][i]['Lat']) == np.float:
                lat.append(js['acList'][i]['Lat'])
            else:
                lat.append(np.nan)
        else:
            lat.append(np.nan)
        if "Long" in js['acList'][i].keys():
            long.append(js['acList'][i]['Long'])
        else:
            long.append(np.nan)


CPU times: user 6min 15s, sys: 9.52 s, total: 6min 25s
Wall time: 10min 47s


In [7]:
df = pd.DataFrame({"DateTime":dt, "id":eyedee, 'Altitude':alt, 'Latitude':lat, 'Longitude':long})

In [8]:
df

,DateTime,id,Altitude,Latitude,Longitude
0,2016-06-20 00:00:00,8721588,31000.0,NaN,NaN
1,2016-06-20 00:00:00,2961715,NaN,0.000000,-2147.483648
2,2016-06-20 00:00:00,10607858,700.0,NaN,NaN
3,2016-06-20 00:00:00,12608926,2775.0,NaN,NaN
4,2016-06-20 00:00:00,7786027,300.0,1.334150,103.987221
...,...,...,...,...,...
6743149,2016-06-20 23:59:00,11401618,6950.0,NaN,NaN
6743150,2016-06-20 23:59:00,11401617,3407.0,NaN,NaN
6743151,2016-06-20 23:59:00,5271422,1000.0,45.481979,13.648616
6743152,2016-06-20 23:59:00,11401619,6752.0,NaN,NaN


In [9]:
# Examine number of records with NULL
df.isnull().sum()

DateTime          0
id                0
Altitude      90618
Latitude     718818
Longitude    718818
dtype: int64

In [10]:
df['DateTime'].value_counts()

2016-06-20 15:43:00    6711
2016-06-20 15:38:00    6694
2016-06-20 14:52:00    6656
2016-06-20 14:44:00    6649
2016-06-20 14:55:00    6648
                       ... 
2016-06-20 04:13:00    2668
2016-06-20 04:06:00    2667
2016-06-20 04:10:00    2643
2016-06-20 04:09:00    2629
2016-06-20 03:07:00     736
Name: DateTime, Length: 1439, dtype: int64

In [11]:
df_select = df.dropna(axis= 0, how= 'any').copy() # Per Dr. Cherry on 6/17/2021

In [12]:
df_select

,DateTime,id,Altitude,Latitude,Longitude
4,2016-06-20 00:00:00,7786027,300.0,1.334150,103.987221
13,2016-06-20 00:00:00,2900289,3752492.0,0.000003,0.000000
23,2016-06-20 00:00:00,4196356,2900.0,51.487190,-0.581210
24,2016-06-20 00:00:00,4494056,25450.0,51.418110,2.221830
25,2016-06-20 00:00:00,11287741,39525.0,50.827541,0.748670
...,...,...,...,...,...
6743117,2016-06-20 23:59:00,8189898,0.0,-33.870198,151.210213
6743128,2016-06-20 23:59:00,12193915,55800.0,37.812332,-120.652634
6743130,2016-06-20 23:59:00,3289137,3159599.0,0.000706,0.000000
6743133,2016-06-20 23:59:00,2894892,2894892.0,0.000000,-2147.483648


In [13]:
# Examine the list of files are already in the output_bucket (i.e., "partly-cloudy-common-area")
conn = boto3.client('s3')
contents = conn.list_objects(Bucket= output_bucket, Prefix= output_subfolder)['Contents']
for f in contents:
    print(f['Key'])

JuneFirst2020_24hr_issr.csv
hourly_issr_summary.csv


In [14]:
outputFileName = 'selectAfileName.csv' # <--------------------------------------------- Specify

wr.s3.to_csv(df_select, f"s3://{output_bucket}/{outputFileName}", index=False)